# Final Project Capestone Part-I & II #


#### "for the *IBM Data Science (Professional) Certification* /  COURSERA" ####

## Table of contents
* [Business Problem](#Problem)
* [Data](#Data)
* [Methodology](#method)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

### Background ###

Jaipur is the capital of India’s Rajasthan state. At the center of its stately street grid (notable in India) stands the opulent, colonnaded City Palace complex. With gardens, courtyards and museums, part of it is still a royal residence.As a capital city it has a good opportunity for an individual/business to open the restaurant/ coffee shop/ cafe near the city center.

### Business Problem <a name='Problem'></a>

This project is focusing on , finding the places where a **restaurant/ coffee shop/ cafe** can be opened.The project will lead to the destinations where there are **few or no restaurants** having the **optimal distance** from the **city center**.This report will be targeted to stakeholders interested in opening an restaurant/ coffee shop/ cafe  in **Jaipur,India**. AS it is a good opportunity to establish a food chain in the capital city.The optimal location will give public advantage as there are few or no restaurants nearby.The city center is seleted because is has a good amount of tourist places,shopping centers,colleges that will help to gather the audience.

### Data <a name='Data'></a>

Gathering the data is the most important factor of data science. More the data more the consistancy in prediction.We will aquire the pincodes of jaipur city from the following https://finkode.com/rj/jaipur.html url.This contans the postal addresses of all the locations in jaipur almost 600 postal locations.We will need only locations that are near to the city center as the outside the city its still a rural area. So we will filter locations according to our need with specific tools.The foursquare will help to find the public places regarding to the all coordinates we have gathered.The following methodology will be used 

We will require the process
* Get all the **picodes** of Jaiupr city**
* Clean the data and will aquire almost **70 picodes near the location**
* Get the **coordinates of pincodes using geopy**
* Use **Foursquare to explore** the needed venues with repect to all the locations we have
* Apply **k-means to get the clusters** having less venues
* Use **Folium to plot the map** wherever required

In [2]:
!pip install geopy
!pip install folium

In [1]:
import pandas as pd
import numpy as np
from project_lib import Project
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy import Nominatim
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

# @hidden_file
project=Project(None,'1fe34bb2-fa56-46b9-9162-6de162ce390e','p-06e18f407e573de9c6743ee3e0e60a9956d5b0b5')

### Gathering the Data ###
Let's first get the data from https://finkode.com/rj/jaipur.html , this contains all the pincodes with the concerned area name

In [3]:
df_pincode=pd.read_html('https://finkode.com/rj/jaipur.html')[0]
df_pincode.head()

,Post Office,District,Pincode
0,A C Jobner S.O,Jaipur,303329
1,Ajmer Road S.O,Jaipur,302006
2,Akhepura B.O,Jaipur,302013
3,Akoda B.O,Jaipur,303338
4,Akodiya B.O,Jaipur,303901


Here the data frame contains Post Office, District and **Picode of Jaipur City**


In [4]:
df_pincode['Address']=df_pincode['Pincode'].astype(str)+' , '+df_pincode['District']+' Rajasthan , India' # Address field to get the latitude and longitudes of the concerned pincode
df_pincode.shape

(589, 4)

As, we can see there are 589 postal areas in jaipur city

## Methodology <a name="method"></a>
In this project we will direct our efforts on detecting areas of jaipur that have low restaurant density. We will limit our analysis to area ~20km around city center.
In first step we have collected the required data: Area pincodes of jaipur city,Now further we will limit the distance to 20km from the city center.
Then we will collect the venues and other details using **FourSquare**
We will present map of all such locations but also create clusters **(using k-means clustering)** of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

### Getting the coordinates of the pincodes using geopy ###
Now, we will fetch the coordinates of the Jaipur City and its areas

In [6]:
cityaddress = ' Jaipur,India'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(cityaddress)
latitude = location.latitude
longitude = location.longitude
address=(latitude,longitude)
print('The geograpical coordinate of Jaipur are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Jaipur are 26.916194, 75.820349.


#### Now the location of areas based on pincodes ####

In [7]:
lati=[]
longi=[]
print("Fetching Location.....")
for pin in df_pincode['Address'].tolist():
    loc=geolocator.geocode(pin)
    if loc is None:
        lati.append('NA')
        longi.append('NA')
    else:
        lati.append(loc.latitude)
        longi.append(loc.longitude)
    print("*", end=' ')
print("Done.")

Fetching Location.....
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *

All set,Add the coordinates to dataframe and check if we have a value with NA , if so then remove it.

#### Now Data cleaning and processing ###
We will perform the operations like add, remove, sort, group to clean our data and visualize it

In [8]:
df_pincode['Latitude']=lati
df_pincode['Longtude']=longi
df_pincode.head()

,Post Office,District,Pincode,Address,Latitude,Longtude
0,A C Jobner S.O,Jaipur,303329,"303329 , Jaipur Rajasthan , India",NA,NA
1,Ajmer Road S.O,Jaipur,302006,"302006 , Jaipur Rajasthan , India",26.9127,75.7739
2,Akhepura B.O,Jaipur,302013,"302013 , Jaipur Rajasthan , India",26.9812,75.7775
3,Akoda B.O,Jaipur,303338,"303338 , Jaipur Rajasthan , India",26.8687,75.4439
4,Akodiya B.O,Jaipur,303901,"303901 , Jaipur Rajasthan , India",26.5928,75.8318


Oh yeah! We can see there are some coordinates not available , we can also observe the picode is so far from the city center

In [9]:
df_pincode=df_pincode[df_pincode['Latitude']!='NA']#Getting the columns having no NA
df_pincode.head()

,Post Office,District,Pincode,Address,Latitude,Longtude
1,Ajmer Road S.O,Jaipur,302006,"302006 , Jaipur Rajasthan , India",26.9127,75.7739
2,Akhepura B.O,Jaipur,302013,"302013 , Jaipur Rajasthan , India",26.9812,75.7775
3,Akoda B.O,Jaipur,303338,"303338 , Jaipur Rajasthan , India",26.8687,75.4439
4,Akodiya B.O,Jaipur,303901,"303901 , Jaipur Rajasthan , India",26.5928,75.8318
5,Alisar B.O,Jaipur,303801,"303801 , Jaipur Rajasthan , India",27.1971,75.5419


First let's have a look how it looks like on the map using **folium**

In [10]:
map_jaipur=folium.Map(location=address,zoom_start=10)
folium.Marker(address,popup='Jaipur').add_to(map_jaipur)
for la,ln in zip(df_pincode['Latitude'],df_pincode['Longtude']):
    folium.CircleMarker(location=[la,ln],radius=5,color='green',fill=True,fill_color='black',fill_opacity=.7).add_to(map_jaipur)
map_jaipur

Looks great having a lots of dot's

Now, Lets remove the duplicates if any

In [13]:
print("Before Duplicate Removal {} rows, cols".format(df_pincode.shape))
df_cleandata=df_pincode.drop_duplicates('Pincode',keep='first') # We have created a new data frame
print("Before Duplicate Removal {} rows, cols".format(df_cleandata.shape))

Before Duplicate Removal (588, 6) rows, cols
Before Duplicate Removal (75, 6) rows, cols


As observed we can see we have 75 unique areas in all ove jaipur

In [14]:
df_cleandata.head()

,Post Office,District,Pincode,Address,Latitude,Longtude
1,Ajmer Road S.O,Jaipur,302006,"302006 , Jaipur Rajasthan , India",26.9127,75.7739
2,Akhepura B.O,Jaipur,302013,"302013 , Jaipur Rajasthan , India",26.9812,75.7775
3,Akoda B.O,Jaipur,303338,"303338 , Jaipur Rajasthan , India",26.8687,75.4439
4,Akodiya B.O,Jaipur,303901,"303901 , Jaipur Rajasthan , India",26.5928,75.8318
5,Alisar B.O,Jaipur,303801,"303801 , Jaipur Rajasthan , India",27.1971,75.5419


we can check the map after updation

In [15]:
map_jaipur=folium.Map(location=address,zoom_start=10)
folium.Marker(address,popup='Jaipur').add_to(map_jaipur)

for la,ln in zip(df_cleandata['Latitude'],df_cleandata['Longtude']):
    folium.CircleMarker(location=[la,ln],radius=5,color='green',fill=True,fill_color='black',fill_opacity=.7).add_to(map_jaipur)
map_jaipur

Now we will calculate distance from the city center assumed the address of the city is the center

In [17]:
# I m using haversine function to calulate distance between the coordinates
import math
distance=[]
def haversine(coord1, latlist,longlist):
    for lat2,lon2 in zip(latlist,longlist):
        
        R = 6372800  # Earth radius in meters
        lat1, lon1 = coord1
        #lat2, lon2 = coord2
    
        phi1, phi2 = math.radians(lat1), math.radians(lat2) 
        dphi       = math.radians(lat2 - lat1)
        dlambda    = math.radians(lon2 - lon1)
    
        a = math.sin(dphi/2)**2 + \
            math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
        distance.append(2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a)))
    return distance

In [18]:
df_cleandata['Center Distance']=haversine(address,df_cleandata['Latitude'],df_cleandata['Longtude'])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [19]:
df_cleandata.head()

,Post Office,District,Pincode,Address,Latitude,Longtude,Center Distance
1,Ajmer Road S.O,Jaipur,302006,"302006 , Jaipur Rajasthan , India",26.9127,75.7739,4623.433971
2,Akhepura B.O,Jaipur,302013,"302013 , Jaipur Rajasthan , India",26.9812,75.7775,8389.081004
3,Akoda B.O,Jaipur,303338,"303338 , Jaipur Rajasthan , India",26.8687,75.4439,37711.901740
4,Akodiya B.O,Jaipur,303901,"303901 , Jaipur Rajasthan , India",26.5928,75.8318,35990.672018
5,Alisar B.O,Jaipur,303801,"303801 , Jaipur Rajasthan , India",27.1971,75.5419,41674.793750


We can see the distance is added , but its in meters so e will convert it to kilometers and rename the column

In [20]:
df_cleandata['Center Distance']=round((df_cleandata['Center Distance']/1000),2)
df_cleandata.rename(columns={'Center Distance':'Center Distance(KM)'},inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [21]:
df_cleandata.head()

,Post Office,District,Pincode,Address,Latitude,Longtude,Center Distance(KM)
1,Ajmer Road S.O,Jaipur,302006,"302006 , Jaipur Rajasthan , India",26.9127,75.7739,4.62
2,Akhepura B.O,Jaipur,302013,"302013 , Jaipur Rajasthan , India",26.9812,75.7775,8.39
3,Akoda B.O,Jaipur,303338,"303338 , Jaipur Rajasthan , India",26.8687,75.4439,37.71
4,Akodiya B.O,Jaipur,303901,"303901 , Jaipur Rajasthan , India",26.5928,75.8318,35.99
5,Alisar B.O,Jaipur,303801,"303801 , Jaipur Rajasthan , India",27.1971,75.5419,41.67


Now we have the distance in kilometers

In [22]:
df_cleandata.shape

(75, 7)

## Analysis <a name="analysis"></a>
Now we will perform the analysis ,we will limit the distance to ~20kms and collect data from foursquaer for the given limits

In [27]:
df_cleandata[df_cleandata['Center Distance(KM)']>20].count()

Post Office            44
District               44
Pincode                44
Address                44
Latitude               44
Longtude               44
Center Distance(KM)    44
dtype: int64

we can observe there are 44 areas wich are more than 20kms so we are removing them

In [30]:
df_clean=df_cleandata[df_cleandata['Center Distance(KM)']<=20]
df_clean.shape

(31, 7)

Now, we have 31 palces around 20kms of distance

In [31]:
# Saving the data for further use
project.save_data(data=df_clean.to_csv(index=False),file_name='jaipurdata.csv',overwrite=True)
print("Dataframe saved as jaipurdata.csv")

Dataframe saved as jaipurdata.csv


## Foursquare in need ##
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category
Category ID corresponding to Food is taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories)
1. we will get the venues nearby the areas we have filtered
2. Food(4d4b7105d754a06374d81259) is used as the category

In [32]:
# The code was removed by Watson Studio for sharing.

In [33]:
# Method that will return the venues with the correspondign data
restaurant='4d4b7105d754a06374d81259'
def get_venues(names,Pincode,latitude,longitude,distance,radius=800):
    venue_list=[] #an empty list is created
    for name,pin,lat,long,dist in zip(names,Pincode,latitude,longitude,distance):
        print(name , end="||")
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long,
            restaurant,
            radius, 
            LIMIT)
        results=requests.get(url).json()['response']['groups'][0]['items']
        venue_list.append([(
            name,
            pin,
            lat,
            long,
             v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name'],dist) for v in results])
        
    nearby=pd.DataFrame([item for venue in venue_list for item in venue])
    nearby.columns=['Area',
                'Pincode',
                'Latitude', 
                'Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude', 
                'Venue Category',
                'Distance(KM)']
    return(nearby)

In [34]:
df_venue=get_venues(df_cleandata['Post Office'],df_cleandata['Pincode'],df_cleandata['Latitude'],
                    df_cleandata['Longtude'],df_cleandata['Center Distance(KM)'])

Ajmer Road S.O||Akhepura B.O||Akoda B.O||Akodiya B.O||Alisar B.O||Amarpura B.O||Amarsar S.O||Amba Bari S.O||Amer Clark Hotel S.O||Amer Road S.O||Amer S.O||Amloda B.O||Anantpura B.O||Anantpura B.O||Anantpura B.O||Andhi S.O||Antela B.O||Arjun Lal Sethi Nagar S.O||Arp Dhelana B.O||Asalpur B.O||Ashok Nagar S.O (Jaipur)||Astikalan B.O||Awania B.O||Badapadampura B.O||Badi Jori B.O||Badwa B.O||Bagawas B.O||Bagwara B.O||Bajaj Nagar S.O||Banar B.O||Bandha Jamwaramgarh B.O||Bani Park S.O||Bansa B.O||Barnagar B.O||Bassi Naga B.O||Bassi S.O (Jaipur)||Beelwadi B.O||Bhadwa B.O||Bhainslana B.O||Bhalloji B.O||Bhambhoria B.O||Bhamod B.O||Bhankrota B.O||Bhato Ki Gali B.O||Bhopawa B.O||Bilanderpur B.O||Bilwa B.O||Bindayaka B.O||Bishangarh B.O||Burthal B.O||Chainpura B.O||Chakwada B.O||Chaturbhuj B.O||Chittora B.O||Chomu S.O||Crpf Campus Lalwas S.O||Dadiya B.O||Dantri B.O||Dhamana B.O||Haldiyon Ka Rasta S.O||Heerapura S.O||High Court S.O (Jaipur)||Jagatpura S.O||Jaitpura Khinchi B.O||Jamdoli S.O||Kalwara 

In [35]:
df_venue.head()

,Area,Pincode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance(KM)
0,Ajmer Road S.O,302006,26.912666,75.773900,Ho-Chi-Minh,26.910030,75.780363,Chinese Restaurant,4.62
1,Ajmer Road S.O,302006,26.912666,75.773900,spice it,26.908134,75.779966,Indian Restaurant,4.62
2,Akhepura B.O,302013,26.981239,75.777517,Sharma Dhaba,26.985141,75.773302,Indian Restaurant,8.39
3,Akhepura B.O,302013,26.981239,75.777517,Khandelwal Dhaba (Purana Wala),26.985036,75.773750,Indian Restaurant,8.39
4,Akhepura B.O,302013,26.981239,75.777517,Khandelwal dhabha,26.985234,75.773903,Diner,8.39


So, looks good. We have the venues their coordinates distance which is less then 20kms etc..

Let's have alook total no of restaurants in a specific location , all are grouped by

In [37]:
df_count=df_venue.groupby(['Area','Latitude','Longitude','Pincode']).size().reset_index(name='No_of_restaurants')
df_count.head()

,Area,Latitude,Longitude,Pincode,No_of_restaurants
0,Ajmer Road S.O,26.912666,75.773900,302006,2
1,Akhepura B.O,26.981239,75.777517,302013,3
2,Amba Bari S.O,26.962243,75.775541,302039,5
3,Amer Clark Hotel S.O,26.846508,75.794000,302018,11
4,Amer Road S.O,26.938614,75.832950,302002,9


In [38]:
df_count.sort_values(by='No_of_restaurants',ascending=False)

,Area,Latitude,Longitude,Pincode,No_of_restaurants
3,Amer Clark Hotel S.O,26.846508,75.794000,302018,11
4,Amer Road S.O,26.938614,75.832950,302002,9
5,Arjun Lal Sethi Nagar S.O,26.890235,75.824723,302004,8
2,Amba Bari S.O,26.962243,75.775541,302039,5
24,Shyam Nagar S.O (Jaipur),26.889447,75.758470,302019,5
8,Bajaj Nagar S.O,26.876235,75.792026,302015,5
18,Jagatpura S.O,26.845418,75.806878,302017,5
10,Bani Park S.O,26.933842,75.794351,302016,5
13,Bhopawa B.O,26.916194,75.820349,303807,4
16,Heerapura S.O,26.897467,75.746716,302021,4


We can observe there is maximum 11 no of restaurants in a location

Lets look into visual form with the pincodes in 1km radius and restaurants in them

In [39]:
cleanmap=folium.Map(location=address,zoom_start=12)
folium.Marker(address,popup='Jaipur').add_to(cleanmap)
for lat,lon in zip(df_clean['Latitude'],df_clean['Longtude']):
     folium.Circle([lat, lon], radius=1000, color='blue', fill=True, fill_color='green', fill_opacity=.3).add_to(cleanmap)
        
for lat,lon,loc in zip(df_venue['Venue Latitude'],df_venue['Venue Longitude'],df_venue['Area']):
    #folium.Marker(location=[lat,lon],Popup=loc).add_to(cleanmap)
    folium.CircleMarker([lat, lon], radius=3, color='red', fill=True, fill_color='green', fill_opacity=1).add_to(cleanmap)
cleanmap

Looks good, and we can see some locations don't have even a single restaurants..

## Clustring of data using K-Means ##
We will use 10 segments to cluster the data and map a grpah to the centers of the k mean
Those locations will be **Prime Location** and can be used for further analysis

In [45]:
number_of_clusters = 7
good_xys = df_clean[['Latitude', 'Longtude']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)
print(kmeans.labels_[0:7])
print(len(kmeans.labels_))
print(kmeans.cluster_centers_)

[1 0 0 5 6 4 6]
31
[[26.95910812 75.78246964]
 [26.89563142 75.73561435]
 [26.77217387 75.82393455]
 [26.90401993 75.93393922]
 [27.0306745  75.9100265 ]
 [26.85302038 75.79386894]
 [26.9126671  75.82470823]]


In [46]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
candidate_area_addresses = []
cluster_centers= [cc for cc in kmeans.cluster_centers_]
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr=lon.astype(str)+","+lat.astype(str)
    location =  geolocator.reverse(addr)
    candidate_area_addresses.append(location.address)    
    print('{}'.format(location.address))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


Addresses of centers of areas recommended for further analysis

Shiv Nagar, Vidyadhar Nagar, Jaipur Municipal Corporation, Jaipur Tehsil, Jaipur, Rajasthan, 302039, India
Tagore Nagar, Badrawas, Jaipur Municipal Corporation, Jaipur Tehsil, Jaipur, Rajasthan, 302021, India
Sanganer Tehsil, Jaipur, Rajasthan, 302022, India
TRatanpura, Bassi Tehsil, Jaipur, Rajasthan, 302031, India
Khoramina, Kukas, Amber Tehsil, Jaipur, Rajasthan, 302028, India
Durgapura, Bapu Nagar, Jaipur Municipal Corporation, Sanganer Tehsil, Jaipur, Rajasthan, 302015, India
Moti Doongri Road, Fateh Tibba, Adarsh Nagar, Jaipur Municipal Corporation, Jaipur Tehsil, Jaipur, Rajasthan, 302004, India


This concludes our analysis. We have created 15 addresses representing centers of zones containing locations with low number of restaurants.
These seven locations have less then 7 restaurants in the corresponding area, so it's good the observe these sites
Although zones are shown on map with a radius of ~1000 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations. Most of the zones are located in Jaipur, which we have identified as interesting due to being popular with tourists, fairly close to city center and well connected by public transport.

## Let's Visualize the final map ##

In [49]:
map_jaipur = folium.Map(location=address, zoom_start=12)
folium.Circle(address, radius=12000, color='red', fill=True, fill_color='green', fill_opacity=.2).add_to(map_jaipur)
for lon,lat in cluster_centers:
    folium.Marker([lon, lat], popup='Interest Area').add_to(map_jaipur) 
for lat, lon in zip(df_venue['Venue Latitude'],df_venue['Venue Longitude']):
    folium.Circle([lat, lon], radius=1000, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.1).add_to(map_jaipur)
map_jaipur

Here the big circle denotes the area of 12 km radius from the center, the poup shows the **Prime Locations** for further analysis having less then 7 restaurnats/cafe/coffeeshops


## Results and Discussion <a name="results"></a>

Our analysis shows that there are good no of restaurants in the range of 20kms, so we can say its a good opputunity for the stakeholder to establish new business.Further we observed that some locations have more than 10 restaurants , means they are in progessing stage.We found 26 locations where the no restaurants or less then 12 restaurants

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 7 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Berlin center but not crowded with existing restaurants. - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify jaipur areas close to center with low number of restaurants in order to aid stakeholders in narrowing down the search for optimal location for a new restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis, and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.